In [21]:
import random

# Draughts Endgame : 4 Kings x 2 Kings

In the end I decided to create a game of drafts following an endgame based on the following:

https://lidraughts.org/study/YiAZbWM6

I decided to go with 4 kings vs 2 kings.

In this scenario, there will be only one 'type' of move. Kings are able to move diagonally forward and backwards, as well as capture other pieces.

## Draughts Class
Class which determines start position of pieces, board state, and display the board

In [24]:
start_positons = random.sample(range(64), 6)
print(start_positons)

[4, 57, 37, 52, 60, 42]


In [35]:
class CurrentBoard:

    BOARD_SIZE = 8
    PLAYER_COUNT = 6

    def __init__(self):
        self.board = [[" " for _ in range(self.BOARD_SIZE)] for _ in range(self.BOARD_SIZE)]
        #self.player_colour = player_colour
        self.state = self.state_of_board()
        self.initial_board()

    # set up the initial board with the 6 pieces in the correct positions
    def initial_board(self):

        start_positons = random.sample(range(self.BOARD_SIZE*self.BOARD_SIZE), self.PLAYER_COUNT)
        white_kings = start_positons[:2]
        black_kings = start_positons[2:]

        for index in white_kings:
            row = index // self.BOARD_SIZE
            col = index % self.BOARD_SIZE
            self.board[row][col] = "W"
        
        # for row, col in black_kings:
        #     self.board[row][col] = "B"

        #print(white_kings)
        #print(black_kings)





    def display_board(self, game_display = False):
        print("+" + "---+" * self.BOARD_SIZE)
        for row in range(self.BOARD_SIZE):
            print("|", end="")
            for col in range(self.BOARD_SIZE):
                print(f" {self.board[row][col]} |", end="")
            print()
            print("+" + "---+" * self.BOARD_SIZE)

    def state_of_board(self):
        # to check the state of the board for a win, loss, or draw
        state = "Unfinished function"
    
        return state

    # In this case the only moves are king moves. They can move as outlined above
    def all_possible_moves(self, player_piece):
        possible_moves = []

        for row in range(self.BOARD_SIZE):
            # need to check all rows dependant on player location
            if self.board[row][0] == " ":
                new_column = self.put_piece_in_column(player_piece, self.board[row])
                new_board = self.board[:row] + [new_column] + self.board[row + 1:]
                possible_moves.append(CurrentBoard(new_board))

        return possible_moves


In [36]:
cb = CurrentBoard()

TypeError: cannot unpack non-iterable int object

In [16]:
"""  taken from searchtreeconnect4 :
https://github.com/Robert-Sheehy/AI/blob/main/2024/Search%20Tree/searchtreeconnect4_24.py
"""
class SearchTreeNode:

    def __init__(self,board_instance,playing_as, ply=0):
        self.children = []
        #self.max_ply_depth = 3
        self.value_is_assigned = False
        self.ply_depth = ply
        self.current_board = board_instance
        self.move_for = playing_as

        if self.current_board.state == "U":
            if self.ply_depth<= self.max_ply_depth:
                self.generate_children()
            else:
                # evaluation function code
                r =1

        else:   # Game over
            if self.current_board.state == "D":
                self.value = 0
            else:
                if ((self.ply_depth % 2) == 0):
                    self.value = -1000
                else:
                    self.value = 1000
            self.value_is_assigned = True

    def min_max_value(self):
        if self.value_is_assigned:
            return self.value

        self.children  = sorted(self.children, key = lambda x:x.min_max_value())

        if ((self.ply_depth % 2) == 0):
            # computers move
            self.value = self.children[-1].value
        else:
            #players move
            self.value = self.children[0].value
        self.value_is_assigned = True

        return self.value

    def generate_children(self):
        for board_for_next_move in self.current_board.all_possible_moves(self.move_for):
            self.children.append(SearchTreeNode(board_for_next_move,self.current_board.other(self.move_for), ply = self.ply_depth +1))


In [17]:
stn = SearchTreeNode(cb, "X")

In [19]:
stn.children

[]

## Play the Game

In [17]:
def play_draughts():

    MAX_MOVES = 10

    response = input("Do you want to play first? (y/n)")
    players_turn = (response == "y")

    response = input("Do you want to play as black or white? (b/w)")
    cb = CurrentBoard()

    player_is_playing = cb.other(cb.other(response))

    for _ in range(MAX_MOVES):

        if players_turn:
            cb.display(game_display=True)

            while True:
                try:
                    choice = int(input("Make your move (Enter index of board): "))
                    if choice < 0 or choice >= 64:
                        print("Invalid index, please try again (0 and 63)")
                    elif cb.board[choice] != ' ':
                        print("Invalid index, the selected position is occupied")
                    else:
                        break
                except ValueError:
                    print("Invalid input, please enter a number")

            cb.move_piece(choice, player_is_playing)
            cb.display()
            
        else:
            search_tree = SearchTreeNode(cb, cb.other(player_is_playing))
            search_tree.min_max_value()
            cb = search_tree.children[-1].current_board
            cb.display()

        if cb.state != "U":
            if cb.state == "D":
                print("It's a draw")
            else:
                if players_turn:
                    print("You win")
                else:
                    print("You lose!")

                cb.display()
            break

        players_turn = not players_turn
